In [1]:
import pandas as pd
from DLC_for_WBFM.utils.projects.utils_project import safe_cd, load_config
from DLC_for_WBFM.utils.pipeline.traces_pipeline import _unpack_configs_for_traces, get_traces_from_3d_tracks
from DLC_for_WBFM.utils.preprocessing.utils_tif import preprocess_all_frames_using_config
import os
from pathlib import Path
import pickle
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

In C:\Users\charles.fieseler\Anaconda3\envs\DLC-for-WBFM\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\charles.fieseler\Anaconda3\envs\DLC-for-WBFM\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\charles.fieseler\Anaconda3\envs\DLC-for-WBFM\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In C:\Users\charles.fieseler\Anaconda3\envs\DLC-for-WBFM\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed tw

In [2]:
project_dir = r"Y:\shared_projects\wbfm\dlc_stacks\Charlie-worm3-long"

with safe_cd(project_dir):
    name = 'neuron4207'
    fname = f"3-tracking/manual_tracking/{name}.csv"
    neuron1 = pd.read_csv(fname, header=[0,1], index_col=0)
    
    name = 'neuron4453'
    fname = f"3-tracking/manual_tracking/{name}.csv"
    neuron2 = pd.read_csv(fname, header=[0,1], index_col=0)
    
    name = 'neuron4528'
    fname = f"3-tracking/manual_tracking/{name}.csv"
    neuron3 = pd.read_csv(fname, header=[0,1], index_col=0)
    
    neuron_zxy = neuron1.join(neuron2)
    neuron_zxy = neuron_zxy.join(neuron3)
    neuron_zxy = neuron_zxy.replace(np.nan, 0.0)
    
    fname = r"3-tracking\manual_tracking\worm3_neurons.xlsx"
    manual_tracks = pd.read_excel(fname, sheet_name=name)
    
    project_cfg = load_config('project_config.yaml')
    
    segment_fname = Path(project_cfg['subfolder_configs']['segmentation'])
    segment_cfg = dict(load_config(segment_fname))
    
    track_fname = Path(project_cfg['subfolder_configs']['tracking'])
    track_cfg = dict(load_config(track_fname))


## See get_traces_from_3d_tracks_using_config

In [3]:
with safe_cd(project_dir):
    _, green_fname, is_mirrored, mask_array, max_dist, num_frames, num_slices, params_start_volume, segmentation_metadata, z_to_xy_ratio = _unpack_configs_for_traces(
            project_cfg, segment_cfg, track_cfg)

In [5]:
with safe_cd(project_dir):
    DEBUG=False
    cfg = project_cfg.copy()
    cfg['preprocessing_config'] = track_cfg['preprocessing_config']
    green_video, _ = preprocess_all_frames_using_config(DEBUG, cfg, verbose=0, vid_fname=green_fname)

  0%|          | 0/1500 [00:00<?, ?it/s]

In [13]:
# Save it!
from pathlib import Path
fname = r"Y:\shared_projects\wbfm\dat\ZIM2051\2021-03-04_16-07-57_worm3_ZIM2051\2021-03-04_16-07-57_worm3_ZIM2051-channel-1-pco_camera2\2021-03-04_16-07-57_worm3_ZIM2051-channel-1-pco_camera2bigtiff.btf"
fname = str(Path(fname).with_suffix('.zarr'))
import zarr
zarr.save_array(fname, green_video)

In [10]:
# Use custom track (one neuron) instead of old ones
DEBUG = False
all_matches, all_neuron_names, green_dat, red_dat = get_traces_from_3d_tracks(DEBUG, neuron_zxy, green_video,
                                                                                  is_mirrored, mask_array, max_dist,
                                                                                  num_frames, num_slices,
                                                                                  params_start_volume,
                                                                                  segmentation_metadata, z_to_xy_ratio)

  0%|▏                                                                                | 4/1500 [00:00<00:38, 39.00it/s]

Matching segmentation and DLC tracking...


  0%|                                                                                         | 0/1500 [00:00<?, ?it/s]

Extracting green traces...


100%|██████████████████████████████████████████████████████████████████████████████| 1500/1500 [08:01<00:00,  3.11it/s]


# Save

In [11]:
with safe_cd(project_dir):
    out_fname = "4-traces/retracked_neurons/green_traces.h5"
    green_dat.to_hdf(out_fname, "df_with_missing")
    
    out_fname = "4-traces/retracked_neurons/red_traces.h5"
    red_dat.to_hdf(out_fname, "df_with_missing")
    
    out_fname = "4-traces/retracked_neurons/matches.pickle"
    with open(out_fname, 'wb') as f:
        pickle.dump(all_matches, f)

C:\Users\charles.fieseler\Anaconda3\envs\DLC-for-WBFM\lib\site-packages\pandas\core\generic.py:2621: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block1_values] [items->MultiIndex([('neuron4207', 'all_values'),
            ('neuron4453', 'all_values'),
            ('neuron4528', 'all_values')],
           names=['neurons', 'data'])]

  encoding=encoding,


# Scratch

In [154]:


def rebuild_hist(frame_df, which_neuron):
    # I accidentally didn't save the full histogram... so I remake it
    frame = segmentation_metadata[0]
    counts, edges = frame.loc[which_neuron,'pixel_counts'], frame.loc[which_neuron,'pixel_values']
    pixel_vals = np.repeat(edges, counts)
    return pixel_vals

167558.0

In [ ]:
rebuild_hist